Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

from keras.datasets import mnist

Load the data

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


Data pre-processing

In [3]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [5]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [6]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

Create model

In [7]:
model=Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    

model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         1

Define training parameters

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        zoom_range = 0.01,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)


train_gen = datagen.flow(X_train, Y_train, batch_size=128)
test_gen = datagen.flow(X_test, Y_test, batch_size=128)

In [10]:
epochs = 10
batch_size = 128
train_steps = X_train.shape[0] // batch_size
valid_steps = X_test.shape[0] // batch_size

es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True,
     )

rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

Train the model

In [11]:
history = model.fit(train_gen, 
                    epochs = epochs, 
                    steps_per_epoch = train_steps,
                    validation_data = test_gen,
                    validation_steps = valid_steps, 
                    callbacks=[es, rp])

Epoch 1/10
468/468 [==============================] - 283s 602ms/step - loss: 0.1263 - accuracy: 0.9604 - val_loss: 0.2998 - val_accuracy: 0.8816 - lr: 0.0010
Epoch 2/10
468/468 [==============================] - 294s 629ms/step - loss: 0.0531 - accuracy: 0.9840 - val_loss: 0.0623 - val_accuracy: 0.9787 - lr: 0.0010
Epoch 3/10
468/468 [==============================] - 278s 595ms/step - loss: 0.0422 - accuracy: 0.9872 - val_loss: 0.0675 - val_accuracy: 0.9793 - lr: 0.0010
Epoch 4/10
468/468 [==============================] - 280s 598ms/step - loss: 0.0391 - accuracy: 0.9879 - val_loss: 0.0343 - val_accuracy: 0.9898 - lr: 0.0010
Epoch 5/10
468/468 [==============================] - 281s 601ms/step - loss: 0.0338 - accuracy: 0.9896 - val_loss: 0.0334 - val_accuracy: 0.9901 - lr: 0.0010
Epoch 6/10
468/468 [==============================] - 282s 603ms/step - loss: 0.0278 - accuracy: 0.9915 - val_loss: 0.0333 - val_accuracy: 0.9900 - lr: 0.0010
Epoch 7/10
468/468 [==========================

Save the model

In [12]:
model.save("model.h5")

In [13]:
!tar -zcvf model.tgz model.h5

a model.h5


In [16]:
conda install -c anaconda pathlib

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\saravanan\anaconda3

  added / updated specs:
    - pathlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pathlib-1.0.1              |     pyhd3eb1b0_1          17 KB  anaconda
    ------------------------------------------------------------
                                           Total:          17 KB

The following NEW packages will be INSTALLED:

  pathlib            anaconda/noarch::pathlib-1.0.1-pyhd3eb1b0_1 None




pathlib-1.0.1        | 17 KB     |            |   0% 
pathlib-1.0.1        | 17 KB     | #########5 |  96% 
pathlib-1.0.1        | 17 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Retrieving notices

In [17]:
conda install -c conda-forge ruamel.yaml

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\saravanan\anaconda3

  added / updated specs:
    - ruamel.yaml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.9.0               |   py38haa244fe_2         982 KB  conda-forge
    ruamel.yaml-0.17.21        |   py38h294d835_1         168 KB  conda-forge
    ruamel.yaml.clib-0.2.6     |   py38h2bbff1b_1         101 KB
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

  ruamel.yaml        conda-forge/win-64::ruamel.yaml-0.17.21-py38h294d835_1 None
  ruamel.yaml.clib   pkgs/main/win-64::ruamel.yaml.clib-0.2.6-py38h2bbff1b_1 None

The following packages will be UPDATED:

  conda                  

Install package

In [18]:
!pip install watson-machine-learning-client

Connect to IBM Watson Machine Learning instance

In [20]:
!pip install -U ibm-watson-machine-learning

  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72555 sha256=eb96bc59f1ce7eea1e4aafcc0a4d2712b73ec1c55b651d2073dbfe0831b443b0
  Stored in directory: c:\users\saravanan\appdata\local\pip\cache\wheels\56\14\78\c2fc4fe36e1e7198e4bde8707cf59b838a7be6859c631142f6
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=500999 sha256=99d2e2bb540f3e053ba99c867d7ce94956927e9e436a260ed3d0f5a800769be3
  Stored in directory: c:\users\saravanan\appdata\local\pip\cache\wheels\54\a5\95\ac2068077b0f41719c2a69a03fdfbdd76ad02cdbb0984ad1a7
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3transfer-2.7.0-py2.py3-none-any.whl size=88603 sha256=8bba709d68977b5ae40fc119977ad4680b5aa5ff3598def13e712c8ec9aeabac
  Stored in directory: c:\users\saravanan\appdata\local\pip\cache\wheels\bf\bf\71\d4b460c36adadb88a24dfd26411c8b4dfa19459d15988c648d
Successfully built ibm-cos-sdk ibm-cos-sdk-core ibm-cos-sdk-s3transfer
  Att

In [24]:
from ibm_watson_machine_learning import APIClient

API_KEY = "Qxwy3byu83al_Lvmk05S2xcRhHqeQiy_4BxWzPcxuB9A"

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [25]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [28]:
#space_uid = guid_from_space_name(client, 'Handwritten Digit Recognition')
#print("Space UID: ", space_uid)
space_uid = guid_from_space_name(client,'hdr')
print("Space UID = " + space_uid)

Space UID = ad4acd34-c854-43ef-8fa2-0e31b3b4a474


In [29]:
client.set.default_space(space_uid)

'SUCCESS'

Define model specifications for deployment

In [30]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [31]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [32]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [33]:
model_id

'9f0258f0-4e81-418f-8925-fad5f717b9ac'

Download the deployed model

In [34]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'C:\\IBM/model.tar.gz'